In [1]:
import os
import matplotlib
import numpy as np
import IPython.display as ipd
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import pandas as pd
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D,Conv1D, Flatten,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense
from keras.optimizers import Adam
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import KFold
import seaborn as sn
import tensorflow.keras as keras
import random

In [2]:
def plot_history(history):
    fig, axs = plt.subplots(2)

    # create accuracy subplot
    axs[0].plot(history.history["accuracy"], label="train accuracy")
    axs[0].plot(history.history["val_accuracy"], label="test accuracy")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc="lower right")
    axs[0].set_title("Accuracy eval")

    # create error subplot
    axs[1].plot(history.history["loss"], label="train error")
    axs[1].plot(history.history["val_loss"], label="test error")
    axs[1].set_ylabel("Error")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc="upper right")
    axs[1].set_title("Error eval")

    plt.show()

In [3]:

# Get train data from csv file
x_train = []
y_train = []
prog = []

nonprog = []

data = pd.read_csv('training_features_10.csv')
print(data.head())            
data = data.drop(['filename'],axis=1)
data = data.drop(['mfcc14'],axis=1)
data = data.drop(['mfcc19'],axis=1)
data = data.drop(['mfcc9'],axis=1)
data = data.drop(['mfcc12'],axis=1)
data = data.drop(['mfcc20'],axis=1)
data = data.drop(['chroma_stft'],axis=1)

data = np.asarray(data)
X = data[:,1:]
y = data[:,0]
y[y=="prog"] = 0
y[y=="non_prog"] = 1



                           filename genre  chroma_stft      rmse  \
0  prog01._The_Same_River.mp3chunk1  prog     0.412844  0.012020   
1  prog01._The_Same_River.mp3chunk2  prog     0.447328  0.016777   
2  prog01._The_Same_River.mp3chunk3  prog     0.549248  0.028440   
3  prog01._The_Same_River.mp3chunk4  prog     0.473921  0.119102   
4  prog01._The_Same_River.mp3chunk5  prog     0.446520  0.128075   

   spectral_centroid  spectral_bandwidth      rolloff  zero_crossing_rate  \
0        2584.011759         3251.498483  4123.208710            12.91667   
1        2778.323385         3335.831973  4928.705386            12.91667   
2        1705.126704         2743.022655  3058.189474            12.91667   
3        1250.770798         2475.443205  2164.361700            12.91667   
4         896.740878         1874.693237  1518.340625            12.91667   

        mfcc1       mfcc2  ...    mfcc11     mfcc12    mfcc13    mfcc14  \
0 -412.344940  146.783371  ...  3.093940  -3.107463 -

In [4]:
X.shape

(19605, 20)

In [5]:
x_train, x_test, y_train, y_test = train_test_split( X, y, test_size=0.5, random_state=42)

X = np.reshape(X, (X.shape[0],1,X.shape[1]))
    

print("X_train shape ",x_train.shape)
print("X_test shape ",y_train.shape)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

y = to_categorical(y)

# Build model using LSTM

input_shape = (1,X.shape[2])
kfold = KFold(n_splits=2, shuffle=True, random_state=45)
batch_size = 32
nb_epochs = 100



X = X.astype('float32')
y = y.astype('float32')



count = 1
for train, test in kfold.split(X, y):
    opt = Adam()
  # Create model
    model = Sequential()
    model.add(keras.layers.LSTM(units=128, dropout=0.01, recurrent_dropout=0.35, return_sequences=True, input_shape=input_shape) )
    model.add(keras.layers.LSTM(units=64, dropout=0.01, recurrent_dropout=0.35, return_sequences=False))
    model.add(Dense(units=16, activation = 'relu'))
    # model.add(keras.layers.Dropout(0.3))
    model.add(Dense(units=8, activation = 'relu'))
    model.add(Dense(units=2, activation='sigmoid'))

    print("Compiling ...")
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    model.summary()

    print("Training ...")
    history = model.fit(X[train], y[train], batch_size=batch_size, epochs=nb_epochs,validation_split=0.1)
    score, accuracy = model.evaluate(X[test], y[test], batch_size=batch_size, verbose=1)
    print("Test loss:  ", score)
    print("Test accuracy:  ", accuracy)

    plot_history(history)

    count += 1
model.save('Model.h5')

X_train shape  (9802, 20)
X_test shape  (9802,)
Compiling ...


/Users/harlow/.local/share/virtualenvs/Harshit-ikHQR9cv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 1, 128)         │        76,288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │            18 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 126,890 (495.66 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 1, 128)         │        76,288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │            18 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 126,890 (495.66 KB)

 Trainable params: 126,890 (495.66 KB)

 Non-trainable params: 0 (0.00 B)

Training ...
Epoch 1/100
276/276 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5825 - loss: 0.6783 - val_accuracy: 0.0000e+00 - val_loss: 0.7883
Epoch 2/100
276/276 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5673 - loss: 0.6807 - val_accuracy: 0.0000e+00 - val_loss: 0.9881
Epoch 3/100
276/276 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5682 - loss: 0.6785 - val_accuracy: 0.0000e+00 - val_loss: 0.9164
Epoch 4/100
276/276 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5644 - loss: 0.6794 - val_accuracy: 0.0000e+00 - val_loss: 0.8861
Epoch 5/100
276/276 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5660 - loss: 0.6790 - val_accuracy: 0.0000e+00 - val_loss: 0.8586
Epoch 6/100
276/276 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5700 - loss: 0.6799 - val_accuracy: 0.0000e+00 - val_loss: 0.8491
Epoch 7/100
276/276 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5688 - loss: 0.6782 - val_accuracy: 0.0000e+00 - val_loss: 0.8646
Epoch 8/100
276/276 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - a